# tinynmc
https://github.com/NillionNetwork/tinynmc

---
Minimal pure-Python implementation of a secure multi-party computation (MPC) protocol for evaluating arithmetic sum-of-products expressions via a non-interactive computation phase.

# Installation and Usage

This library is available as a package on PyPI:

In [4]:
! python -m pip install tinynmc

The library can be imported in the usual way:

In [6]:
import tinynmc
from tinynmc import *

# Basic Example

This example involves three contributors a, b, and c (parties submitting private input values) and three nodes 0, 1, and 2 (parties performing a computation):



In [7]:
nodes = [node(), node(), node()]


The overall sum-of-products expression being computed is (1 * 2 * 3) + (4 * 5). First, the contributors agree on a workflow signature. The signature lists the number of factors in each term:



In [8]:
signature = [3, 2]

The signature must be shared with every node so that the nodes can collectively perform the preprocessing phase; this can be accomplished using any MPC protocol that supports multiplication of secret-shared values, such as the SPDZ protocol (a similarly simple implementation of which can be seen in the TinySMPC library):

>

In [9]:
preprocess(signature, nodes)

Next, each factor in the workflow is contributed by one of three contributors a, b, or c, with the ownership pattern `(a * b * c) + (a * b)`. Each factor is referenced by the contributors according to its `(term_index, factor_index)` coordinate within the overall expression: `((0, 0) * (0, 1)) + ((1, 0) * (1, 1) * (1, 2))`.

Each contributor can convert its coordinate-value pairs into masked factors by (1) requesting the multiplicative shares of the masks for each coordinate, and (2) masking its factors at each coordinate using those masks:

>

In [10]:
coords_to_values_a = {(0, 0): 1, (1, 0): 4}
masks_from_nodes_a = [node.masks(coords_to_values_a.keys()) for node in nodes]
masked_factors_a = masked_factors(coords_to_values_a, masks_from_nodes_a)

coords_to_values_b = {(0, 1): 2, (1, 1): 5}
masks_from_nodes_b = [node.masks(coords_to_values_b.keys()) for node in nodes]
masked_factors_b = masked_factors(coords_to_values_b, masks_from_nodes_b)

coords_to_values_c = {(0, 2): 3}
masks_from_nodes_c = [node.masks(coords_to_values_c.keys()) for node in nodes]
masked_factors_c = masked_factors(coords_to_values_c, masks_from_nodes_c)


Each contributor then broadcasts all of its masked factors to every node, so every node receives all of the masked factors from all of the contributors:


In [11]:
broadcast = [masked_factors_a, masked_factors_b, masked_factors_c]



Then, every node can locally compute its share of the overall result:


In [12]:
result_share_at_node_0 = nodes[0].compute(signature, broadcast)
result_share_at_node_1 = nodes[1].compute(signature, broadcast)
result_share_at_node_2 = nodes[2].compute(signature, broadcast)


Finally, the result can be reconstructed via summation from the result shares received from the nodes:


In [13]:
int(sum([result_share_at_node_0, result_share_at_node_1, result_share_at_node_2]))

26